# InstructVault + OpenAI (Google Colab)

This notebook renders a prompt with InstructVault and sends it to the OpenAI API.

**Note:** Keep your API key private. Do not commit it to source control.

In [ ]:
!pip -q install instructvault openai

In [ ]:
import os, getpass

if 'OPENAI_API_KEY' not in os.environ:
    os.environ['OPENAI_API_KEY'] = getpass.getpass('Enter OpenAI API key: ')
print('API key set:', 'OPENAI_API_KEY' in os.environ)

In [ ]:
from pathlib import Path

repo_path = '/content/ivault_demo'
repo = Path(repo_path)
(repo / 'prompts').mkdir(parents=True, exist_ok=True)

prompt_text = '''spec_version: "1.0"
name: hello_world
variables:
  required: [name]
messages:
  - role: system
    content: "You are a helpful assistant."
  - role: user
    content: "Say hello to {{ name }}."
tests:
  - name: includes_name
    vars: { name: "Ava" }
    assert: { contains_any: ["Ava"] }
'''

(repo / 'prompts' / 'hello_world.prompt.yml').write_text(prompt_text, encoding='utf-8')
print('Prompt written:', repo / 'prompts' / 'hello_world.prompt.yml')

In [ ]:
from instructvault import InstructVault

vault = InstructVault(repo_root=repo_path)
messages = vault.render('prompts/hello_world.prompt.yml', vars={'name':'Colab'})
messages

In [ ]:
from openai import OpenAI

client = OpenAI()
response = client.chat.completions.create(
    model='gpt-4.1',
    messages=messages,
)
print(response.choices[0].message.content)